In [ ]:
  import pandas as pd
  import numpy as np
  import plotly.express as px
  import plotly.graph_objects as go
  import folium
  from folium.plugins import HeatMap
  from folium.plugins import MarkerCluster
  import requests
  import random

In [ ]:
est_url = 'https://raw.githubusercontent.com/angelnmara/geojson/master/mexicoHigh.json' 
mun_url = 'https://raw.githubusercontent.com/angelnmara/geojson/master/MunicipiosMexico.json'
# Archivo GeoJSON
est_regions = requests.get(est_url).json()
mun_regions = requests.get(mun_url).json()

In [ ]:
# función 
def make_dataframe_states(geo_data):
  '''
  Función para obtener un dataframe de estados
  '''
  # get features
  features = geo_data['features']
  # get country, state and mun
  country = []
  state = []

  # loop for each feature
  for feature in features:
    country.append(feature['properties']['CNTRY'])
    state.append(feature['properties']['name'])
    #mun.append(feature['properties']['NAME_2'])
  # dataframe
  df2 = pd.DataFrame([country, state]).T
  df2.columns = ['country', 'state']
  # random percentage
  df2['percentage'] = [random.uniform(0, 1) for i in range(df2.shape[0])]

  return df2

In [ ]:
# función 
def make_dataframe_mun(geo_data):
  '''
  Función para obtener un dataframe de estados y municipios 
  '''
  # get features
  features = geo_data['features']
  # get country, state and mun
  country = []
  state = []
  mun = []
  # loop for each feature
  for feature in features:
    country.append(feature['properties']['NAME_0'])
    state.append(feature['properties']['NAME_1'])
    mun.append(feature['properties']['NAME_2'])
  # dataframe
  df2 = pd.DataFrame([country, state, mun]).T
  df2.columns = ['country', 'state', 'mun']
  # random percentage
  df2['percentage'] = [random.uniform(0, 1) for i in range(df2.shape[0])]
  
  return df2

In [ ]:
df_est = make_dataframe_states(est_regions)
df_est.head()

,country,state,percentage
0,Mexico,Zacatecas,0.355422
1,Mexico,Yucatán,0.567715
2,Mexico,Veracruz,0.367857
3,Mexico,Tlaxcala,0.777042
4,Mexico,Tamaulipas,0.111107


In [ ]:
df_mun = make_dataframe_mun(mun_regions)
df_mun.head()

,country,state,mun,percentage
0,Mexico,Aguascalientes,Aguascalientes,0.263064
1,Mexico,Aguascalientes,Asientos,0.730717
2,Mexico,Aguascalientes,Calvillo,0.948170
3,Mexico,Aguascalientes,Cosío,0.576196
4,Mexico,Aguascalientes,Jesús María,0.684880


# **Bases producción**

In [ ]:
# nombres de columnas
colnames = ['Anio', 'Idestado', 'Nomestado', 'Idddr', 'Nomddr', 'Idcader',
       'Nomcader', 'Idmunicipio', 'Nommunicipio', 'Idciclo',
       'Nomcicloproductivo', 'Idmodalidad', 'Nommodalidad', 'Idunidadmedida',
       'Nomunidad', 'Idcultivo', 'Nomcultivo', 'Sembrada', 'Cosechada',
       'Siniestrada', 'Volumenproduccion', 'Rendimiento', 'Precio',
       'Valorproduccion']
selected_columns = ['Anio', 'Idestado', 'Nomestado', 'Idmunicipio',	'Nommunicipio', 'Nomunidad', 'Nomcultivo', 'Volumenproduccion']
# leemos bases y cambiamos nombres de columnas para homologar nombres    
#  base 2019   
df_prod_2019 = pd.read_csv('Cierre_agricola_mun_2019.csv',  encoding = "ISO-8859-1")
df_prod_2019.columns = colnames
df_prod_2019 = df_prod_2019[selected_columns]
#  base 2020
df_prod_2020 = pd.read_csv('Cierre_agricola_mun_2020.csv',  encoding = "ISO-8859-1")
df_prod_2020.columns = colnames
df_prod_2020 = df_prod_2020[selected_columns]
#  base 2021
df_prod_2021 = pd.read_csv('Cierre_agr_mun_2021.csv',  encoding = "ISO-8859-1")
df_prod_2021.columns = colnames
df_prod_2021 = df_prod_2021[selected_columns]
# producción de leche 2021
df_prod_leche = pd.read_excel('ProdLeche.xlsx')
df_prod_leche = df_prod_leche.drop('EML', axis=1)
df_prod_leche['Idestado'] = np.nan
df_prod_leche['Idmunicipio'] = np.nan
df_prod_leche['Nomunidad'] = 'Miles de lts'
df_prod_leche['Nomcultivo'] = 'Leche'
df_prod_leche = df_prod_leche[['Año', 'Idestado', 'Estado', 'Idmunicipio',	'Municipio', 'Nomunidad', 'Nomcultivo', 'Prod (Miles de lts)']]
df_prod_leche.columns = selected_columns

En este paso se unen las bases por filas 

In [ ]:
df_prod = pd.concat([df_prod_2019, df_prod_2020, df_prod_2020, df_prod_leche], axis=0)
df_prod.tail()

,Anio,Idestado,Nomestado,Idmunicipio,Nommunicipio,Nomunidad,Nomcultivo,Volumenproduccion
4308,2020,NaN,Zacatecas,NaN,Villa Hidalgo,Miles de lts,Leche,1575.448
4309,2020,NaN,Zacatecas,NaN,Villanueva,Miles de lts,Leche,789.592
4310,2020,NaN,Zacatecas,NaN,Zacatecas,Miles de lts,Leche,2833.526
4311,2020,NaN,Zacatecas,NaN,Trancoso,Miles de lts,Leche,1775.377
4312,2020,NaN,Zacatecas,NaN,Santa María de La Paz,Miles de lts,Leche,149.191


Filtramos los productos en el campo **Nomcultivo**:
- Frijol : Frijol, Frijol forrajero, Frijol x pelón, Semilla de frijol
- Arroz : Arroz palay
- Maíz : Maíz forrajero en verde, Maíz grano, Maíz palomero, Semilla de maíz grano
- Leche : Leche
- Trigo : Trigo forrajero verde, Trigo grano, Trigo ornamental, Semilla de trigo grano

In [ ]:
prod_filt = {
    'Frijol': 'Frijol', 
    'Frijol forrajero': 'Frijol', 
    'Frijol x pelón': 'Frijol', 
    'Semilla de frijol': 'Frijol',
    'Arroz palay' : 'Arroz',
    'Maíz forrajero en verde': 'Maíz', 
    'Maíz grano': 'Maíz', 
    'Maíz palomero': 'Maíz', 
    'Semilla de maíz grano': 'Maíz',
    'Leche': 'Leche',
    'Trigo forrajero verde': 'Trigo', 
    'Trigo grano': 'Trigo', 
    'Trigo ornamental': 'Trigo', 
    'Semilla de trigo grano': 'Trigo'}

In [ ]:
# se crea nuevo campo con nombres de productos únicos
df_prod['Producto'] = df_prod['Nomcultivo'].map(prod_filt)
# se filtra por llaves únicas 
df_prod = df_prod[df_prod['Producto'].isin(set(prod_filt.values()))]
df_prod.head()

,Anio,Idestado,Nomestado,Idmunicipio,Nommunicipio,Nomunidad,Nomcultivo,Volumenproduccion,Producto
8,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Maíz forrajero en verde,244746.02,Maíz
9,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Maíz grano,4151.04,Maíz
16,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Frijol,109.04,Frijol
17,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Maíz forrajero en verde,48662.50,Maíz
18,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Maíz grano,624.34,Maíz


In [ ]:
df_prod[df_prod['Nomcultivo']=='Frijol forrajero']

,Anio,Idestado,Nomestado,Idmunicipio,Nommunicipio,Nomunidad,Nomcultivo,Volumenproduccion,Producto
15847,2019,16.0,Michoacán,99.0,Tuzantla,Tonelada,Frijol forrajero,14000.0,Frijol


In [ ]:
df_prod[df_prod['Nomcultivo']=='Leche']

,Anio,Idestado,Nomestado,Idmunicipio,Nommunicipio,Nomunidad,Nomcultivo,Volumenproduccion,Producto
0,2021,NaN,Aguascalientes,NaN,Aguascalientes,Miles de lts,Leche,101236.652,Leche
1,2021,NaN,Aguascalientes,NaN,Asientos,Miles de lts,Leche,40735.343,Leche
2,2021,NaN,Aguascalientes,NaN,Calvillo,Miles de lts,Leche,30311.634,Leche
3,2021,NaN,Aguascalientes,NaN,Cosío,Miles de lts,Leche,27769.197,Leche
4,2021,NaN,Aguascalientes,NaN,Jesús María,Miles de lts,Leche,50049.518,Leche
...,...,...,...,...,...,...,...,...,...
4308,2020,NaN,Zacatecas,NaN,Villa Hidalgo,Miles de lts,Leche,1575.448,Leche
4309,2020,NaN,Zacatecas,NaN,Villanueva,Miles de lts,Leche,789.592,Leche
4310,2020,NaN,Zacatecas,NaN,Zacatecas,Miles de lts,Leche,2833.526,Leche
4311,2020,NaN,Zacatecas,NaN,Trancoso,Miles de lts,Leche,1775.377,Leche


# Base a nivel entidad

In [ ]:
df_entidad = pd.merge(df_prod, df_est.drop('country', axis=1),  
                      left_on=['Nomestado'], 
                      right_on=['state'], how='outer')

df_entidad.head()

,Anio,Idestado,Nomestado,Idmunicipio,Nommunicipio,Nomunidad,Nomcultivo,Volumenproduccion,Producto,state,percentage
0,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Maíz forrajero en verde,244746.02,Maíz,Aguascalientes,0.098049
1,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Maíz grano,4151.04,Maíz,Aguascalientes,0.098049
2,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Frijol,109.04,Frijol,Aguascalientes,0.098049
3,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Maíz forrajero en verde,48662.50,Maíz,Aguascalientes,0.098049
4,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Maíz grano,624.34,Maíz,Aguascalientes,0.098049


In [ ]:
df_states_reference = df_entidad.groupby(['Anio','state', 'Producto', 'Nomunidad']).agg({'Volumenproduccion': np.sum}).reset_index()
df_states_reference.columns = ['Anio', 'Entidad', 'Producto', 'Nomunidad', 'Volumenproduccion']

In [ ]:
df_states_reference.tail()

,Anio,Entidad,Producto,Nomunidad,Volumenproduccion
248,2021,Tabasco,Leche,Miles de lts,105101.180
249,2021,Tamaulipas,Leche,Miles de lts,20815.269
250,2021,Tlaxcala,Leche,Miles de lts,81467.295
251,2021,Yucatán,Leche,Miles de lts,2157.093
252,2021,Zacatecas,Leche,Miles de lts,188787.511


In [ ]:
df_states_reference[df_states_reference['Producto']=='Maíz']

,Anio,Entidad,Producto,Nomunidad,Volumenproduccion
1,2019,Aguascalientes,Maíz,Tonelada,1168390.41
2,2019,Baja California,Maíz,Tonelada,49487.85
5,2019,Baja California Sur,Maíz,Tonelada,62179.93
9,2019,Campeche,Maíz,Tonelada,450581.23
12,2019,Chiapas,Maíz,Tonelada,1255419.51
...,...,...,...,...,...
207,2020,Tamaulipas,Maíz,Tonelada,1178778.58
211,2020,Tlaxcala,Maíz,Tonelada,1254650.42
215,2020,Veracruz,Maíz,Tonelada,2674443.34
219,2020,Yucatán,Maíz,Tonelada,200443.00


In [ ]:
df_states_reference.to_csv('produccion_estados.csv', index=False,  encoding="utf-8-sig")